# KuCoin Updater

In [ ]:
import pandas as pd
import numpy as np


import os
import requests
import json
import base64

import time
import base64
import hmac
import hashlib

## Run this to append KuCoin Data file (holdings)

In [ ]:
response = requests.request('get', url, headers=headers)

In [7]:
def kucoin_append():
    
    # First importing the needed libraries:

    import pandas as pd
    import numpy as np


    import os
    import requests
    import json
    import base64
    import time
    import base64
    import hmac
    import hashlib

    # Now importing the env file so the script can access the KuCoin API keys:
    import env

    # Defining the api keys with their own variables:
    api_key = env.kc_api_key
    api_s = env.kc_secret_api
    api_pp = env.kc_passphrase
    api_uid = env.kc_uid

    # creating the api keys for use in the calls:
    api_key = env.kc_api_key
    api_secret = env.kc_secret_api
    api_passphrase = env.kc_passphrase
    url = 'https://api.kucoin.com/api/v1/accounts'
    now = int(time.time() * 1000)
    str_to_sign = str(now) + 'GET' + '/api/v1/accounts'
    signature = base64.b64encode(
    hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
    "KC-API-SIGN": signature,
    "KC-API-TIMESTAMP": str(now),
    "KC-API-KEY": api_key,
    "KC-API-PASSPHRASE": passphrase,
    "KC-API-KEY-VERSION": str(2)
    }

    # Getting the base response with the top level account values:
    response = requests.request('get', url, headers=headers)


    # Creating the account dataframe using the response request I just created:
    df = pd.DataFrame.from_dict(response.json()['data'])

    # Column cleanup:
    df.drop(columns = 'id', inplace = True)

    # Getting prices for coins:
    coin_list = df['currency'].unique().tolist()

    # USDC and USDT don't work in this list because they are "equivalent" of USD, so it comes back as a NoneType, leading to a none-type error later if I don't remove them from the list at this point.
    coin_list.remove('USDC')
    coin_list.remove('USDT')
    
    # This for loop will create a list of prices by calling each crypto within my 'coin_list' list. 

    price_list = []
    for coin in coin_list:
        prices = float(requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={coin}-USDT').json()['data']['price'])
    #     print(prices), print(type(prices))
        price_list.append(prices)

    # Now creating a dictionary of the coin prices:
    coin_dict = {"coin":coin_list, "price":price_list}

    # Dataframe from the dictionary:
    df_prices = pd.DataFrame(coin_dict)

    # creating a copy dataframe of the acct info (this step can be dropped in future)
    account = df[df['type'] == 'trade'].copy()
    account = account.reset_index(drop = True)
    account.rename(columns = {'type': "act_name"}, inplace = True)


    # Now should be able to join the two dataframes. I have to join the two dataframes first before I can multiply columns to create the "$ value" column which is the end goal here.

    # Merge, inner join:

    holdings_append = account.merge(df_prices, left_on = 'currency', right_on = 'coin', how = 'left')

    # Now changing the value types of the columns with numbers in them from objects to float64:
    holdings_append['balance'] = holdings_append.balance.astype(float)
    holdings_append['available'] = holdings_append.available.astype(float)
    holdings_append['holds'] = holdings_append.holds.astype(float)

    # Now working through adding calculated columns that I'll later select from if it doesn't return 'nan':

    holdings_append['value_tmp'] = round(holdings_append.price * holdings_append.balance, 2)

    holdings_append["dollar_value"] = np.where(holdings_append['value_tmp'].notnull(), holdings_append['value_tmp'], holdings_append['balance'])
    holdings_append.dollar_value = holdings_append.dollar_value.round(2)

    # Dropping extra columns:
    holdings_append.drop(columns = ['coin', 'value_tmp'], inplace = True)

    # Adding date column, and changing date to the dataframe index:
    # holdings.insert(0, 'date', pd.to_datetime('today').strftime('%Y-%m-%d'))
    holdings_append.insert(0, 'date', pd.to_datetime('now').replace(microsecond=0))
    holdings_append.date = pd.to_datetime(holdings_append.date)
    holdings_append = holdings_append.set_index('date').sort_index()

    # printing out result, and saving to csv and Excel.
    return holdings_append

    # holdings.to_csv("account_holdings_index_test.csv")
    # holdings.to_csv("account_holdings_no_index.csv", index = False)


    # holdings.to_excel("account_holdings_index.xlsx")
    # holdings.to_excel("account_holdings_no_index.xlsx", index = False)
    # print("CSV written successfully.")

In [8]:
kucoin_append()

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-03 13:34:15,USDT,trade,2319.402655,1969.440630,349.962025,NaN,2319.40
2021-09-03 13:34:15,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-03 13:34:15,BTC,trade,0.013654,0.013654,0.000000,50770.4000,693.24
2021-09-03 13:34:15,BTC3S,trade,342.742100,120.711700,222.030400,1.8036,618.17
2021-09-03 13:34:15,BTC3L,trade,128.540200,21.594900,106.945300,1.3262,170.47
2021-09-03 13:34:15,ATOM,trade,0.236600,0.236600,0.000000,23.7665,5.62
2021-09-03 13:34:15,ALGO,trade,0.000000,0.000000,0.000000,1.2377,0.00
2021-09-03 13:34:15,ETH,trade,0.000000,0.000000,0.000000,3952.3500,0.00
2021-09-03 13:34:15,MATIC,trade,0.000000,0.000000,0.000000,1.4732,0.00


In [32]:
# First importing the needed libraries:

import pandas as pd
import numpy as np


import os
import requests
import json
import base64
import time
import base64
import hmac
import hashlib

# Now importing the env file so the script can access the KuCoin API keys:
import env

# Defining the api keys with their own variables:
api_key = env.kc_api_key
api_s = env.kc_secret_api
api_pp = env.kc_passphrase
api_uid = env.kc_uid

# creating the api keys for use in the calls:
api_key = env.kc_api_key
api_secret = env.kc_secret_api
api_passphrase = env.kc_passphrase
url = 'https://api.kucoin.com/api/v1/accounts'
now = int(time.time() * 1000)
str_to_sign = str(now) + 'GET' + '/api/v1/accounts'
signature = base64.b64encode(
hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
headers = {
"KC-API-SIGN": signature,
"KC-API-TIMESTAMP": str(now),
"KC-API-KEY": api_key,
"KC-API-PASSPHRASE": passphrase,
"KC-API-KEY-VERSION": str(2)
}

# Getting the base response with the top level account values:
response = requests.request('get', url, headers=headers)


# Creating the account dataframe using the response request I just created:
df = pd.DataFrame.from_dict(response.json()['data'])

# Column cleanup:
df.drop(columns = 'id', inplace = True)

# Getting prices for coins:

coin_list = ['BTC', 'ETH', 'ADA', 'ALGO', 'ATOM', 'XLM', 'DOT']

# This for loop will create a list of prices by calling each crypto within my 'coin_list' list. 

price_list = []
for coin in coin_list:
    prices = float(requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={coin}-USDT').json()['data']['price'])
#     print(prices), print(type(prices))
    price_list.append(prices)

# Now creating a dictionary of the coin prices:
coin_dict = {"coin":coin_list, "price":price_list}

# Dataframe from the dictionary:
df_prices = pd.DataFrame(coin_dict)

# creating a copy dataframe of the acct info (this step can be dropped in future)
account = df[df['type'] == 'trade'].copy()
account = account.reset_index(drop = True)
account.rename(columns = {'type': "act_name"}, inplace = True)


# Now should be able to join the two dataframes. I have to join the two dataframes first before I can multiply columns to create the "$ value" column which is the end goal here.

# Merge, inner join:

holdings_append = account.merge(df_prices, left_on = 'currency', right_on = 'coin', how = 'left')

# Now changing the value types of the columns with numbers in them from objects to float64:
holdings_append['balance'] = holdings_append.balance.astype(float)
holdings_append['available'] = holdings_append.available.astype(float)
holdings_append['holds'] = holdings_append.holds.astype(float)

# Now working through adding calculated columns that I'll later select from if it doesn't return 'nan':

holdings_append['value_tmp'] = round(holdings_append.price * holdings_append.balance, 2)

holdings_append["dollar_value"] = np.where(holdings_append['value_tmp'].notnull(), holdings_append['value_tmp'], holdings_append['balance'])
holdings_append.dollar_value = holdings_append.dollar_value.round(2)

# Dropping extra columns:
holdings_append.drop(columns = ['coin', 'value_tmp'], inplace = True)

# Adding date column, and changing date to the dataframe index:
# holdings.insert(0, 'date', pd.to_datetime('today').strftime('%Y-%m-%d'))
holdings_append.insert(0, 'date', pd.to_datetime('now').replace(microsecond=0))
holdings_append.date = pd.to_datetime(holdings_append.date)
holdings_append = holdings_append.set_index('date').sort_index()

# printing out result, and saving to csv and Excel.
holdings_append

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-03 15:43:44,USDT,trade,2033.431652,1683.469627,349.962025,NaN,2033.43
2021-09-03 15:43:44,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-03 15:43:44,BTC,trade,0.013654,0.013654,0.000000,50607.0000,691.01
2021-09-03 15:43:44,BTC3S,trade,342.742100,120.711700,222.030400,NaN,342.74
2021-09-03 15:43:44,BTC3L,trade,343.540200,236.594900,106.945300,NaN,343.54
2021-09-03 15:43:44,ATOM,trade,0.236600,0.236600,0.000000,24.2138,5.73
2021-09-03 15:43:44,ALGO,trade,0.000000,0.000000,0.000000,1.2303,0.00
2021-09-03 15:43:44,ETH,trade,0.000000,0.000000,0.000000,3966.7800,0.00
2021-09-03 15:43:44,MATIC,trade,0.000000,0.000000,0.000000,NaN,0.00


In [33]:
holdings_append

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-03 15:43:44,USDT,trade,2033.431652,1683.469627,349.962025,NaN,2033.43
2021-09-03 15:43:44,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-03 15:43:44,BTC,trade,0.013654,0.013654,0.000000,50607.0000,691.01
2021-09-03 15:43:44,BTC3S,trade,342.742100,120.711700,222.030400,NaN,342.74
2021-09-03 15:43:44,BTC3L,trade,343.540200,236.594900,106.945300,NaN,343.54
2021-09-03 15:43:44,ATOM,trade,0.236600,0.236600,0.000000,24.2138,5.73
2021-09-03 15:43:44,ALGO,trade,0.000000,0.000000,0.000000,1.2303,0.00
2021-09-03 15:43:44,ETH,trade,0.000000,0.000000,0.000000,3966.7800,0.00
2021-09-03 15:43:44,MATIC,trade,0.000000,0.000000,0.000000,NaN,0.00


In [3]:
# THIS CODE works to append to the csv file itself:

holdings_append.to_csv('data.csv', mode = 'a', header = False)

In [4]:
holdings_append.shape

(9, 7)

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(credentials)

spreadsheet = client.open('Crypto-Data')

with open('data.csv', 'r') as file_obj:
    content = file_obj.read()
    client.import_csv(spreadsheet.id, data=content)

## Full Function start to finish

In [12]:
def kucoin_append():
    
    # First importing the needed libraries:

    import pandas as pd
    import numpy as np


    import os
    import requests
    import json
    import base64
    import time
    import base64
    import hmac
    import hashlib

    # Now importing the env file so the script can access the KuCoin API keys:
    import env

    # Defining the api keys with their own variables:
    api_key = env.kc_api_key
    api_s = env.kc_secret_api
    api_pp = env.kc_passphrase
    api_uid = env.kc_uid

    # creating the api keys for use in the calls:
    api_key = env.kc_api_key
    api_secret = env.kc_secret_api
    api_passphrase = env.kc_passphrase
    url = 'https://api.kucoin.com/api/v1/accounts'
    now = int(time.time() * 1000)
    str_to_sign = str(now) + 'GET' + '/api/v1/accounts'
    signature = base64.b64encode(
    hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
    "KC-API-SIGN": signature,
    "KC-API-TIMESTAMP": str(now),
    "KC-API-KEY": api_key,
    "KC-API-PASSPHRASE": passphrase,
    "KC-API-KEY-VERSION": str(2)
    }

    # Getting the base response with the top level account values:
    response = requests.request('get', url, headers=headers)


    # Creating the account dataframe using the response request I just created:
    df = pd.DataFrame.from_dict(response.json()['data'])

    # Column cleanup:
    df.drop(columns = 'id', inplace = True)

    # Getting prices for coins:
    coin_list = df['currency'].unique().tolist()

    # USDC and USDT don't work in this list because they are "equivalent" of USD, so it comes back as a NoneType, leading to a none-type error later if I don't remove them from the list at this point.
    coin_list.remove('USDC')
    coin_list.remove('USDT')
    
    # This for loop will create a list of prices by calling each crypto within my 'coin_list' list. 

    price_list = []
    for coin in coin_list:
        prices = float(requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={coin}-USDT').json()['data']['price'])
    #     print(prices), print(type(prices))
        price_list.append(prices)

    # Now creating a dictionary of the coin prices:
    coin_dict = {"coin":coin_list, "price":price_list}

    # Dataframe from the dictionary:
    df_prices = pd.DataFrame(coin_dict)

    # creating a copy dataframe of the acct info (this step can be dropped in future)
    account = df[df['type'] == 'trade'].copy()
    account = account.reset_index(drop = True)
    account.rename(columns = {'type': "act_name"}, inplace = True)


    # Now should be able to join the two dataframes. I have to join the two dataframes first before I can multiply columns to create the "$ value" column which is the end goal here.

    # Merge, inner join:

    holdings_append = account.merge(df_prices, left_on = 'currency', right_on = 'coin', how = 'left')

    # Now changing the value types of the columns with numbers in them from objects to float64:
    holdings_append['balance'] = holdings_append.balance.astype(float)
    holdings_append['available'] = holdings_append.available.astype(float)
    holdings_append['holds'] = holdings_append.holds.astype(float)

    # Now working through adding calculated columns that I'll later select from if it doesn't return 'nan':

    holdings_append['value_tmp'] = round(holdings_append.price * holdings_append.balance, 2)

    holdings_append["dollar_value"] = np.where(holdings_append['value_tmp'].notnull(), holdings_append['value_tmp'], holdings_append['balance'])
    holdings_append.dollar_value = holdings_append.dollar_value.round(2)

    # Dropping extra columns:
    holdings_append.drop(columns = ['coin', 'value_tmp'], inplace = True)

    # Adding date column, and changing date to the dataframe index:
    # holdings.insert(0, 'date', pd.to_datetime('today').strftime('%Y-%m-%d'))
    holdings_append.insert(0, 'date', pd.to_datetime('now').replace(microsecond=0))
    holdings_append.date = pd.to_datetime(holdings_append.date)
    holdings_append = holdings_append.set_index('date').sort_index()

    # printing out result, and saving to csv and Excel.
    return holdings_append

    # holdings.to_csv("account_holdings_index_test.csv")
    # holdings.to_csv("account_holdings_no_index.csv", index = False)


    # holdings.to_excel("account_holdings_index.xlsx")
    # holdings.to_excel("account_holdings_no_index.xlsx", index = False)
    # print("CSV written successfully.")
    
holdings_append = kucoin_append()   


# THIS CODE works to append to the csv file itself. Only works if there is an existing file. I will add the if statement later that'll check if a file exists and create a new one, or use the append function:
holdings_append.to_csv('data.csv', mode = 'a', header = False)

# Uploading to Google sheets

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(credentials)

spreadsheet = client.open('Crypto-Data')

with open('data.csv', 'r') as file_obj:
    content = file_obj.read()
    client.import_csv(spreadsheet.id, data=content)

In [13]:
kucoin_append()

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-03 14:11:35,USDT,trade,2319.402655,1969.440630,349.962025,NaN,2319.40
2021-09-03 14:11:35,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-03 14:11:35,BTC,trade,0.013654,0.013654,0.000000,50690.8000,692.15
2021-09-03 14:11:35,BTC3S,trade,342.742100,120.711700,222.030400,1.8080,619.68
2021-09-03 14:11:35,BTC3L,trade,128.540200,21.594900,106.945300,1.3224,169.98
2021-09-03 14:11:35,ATOM,trade,0.236600,0.236600,0.000000,23.7548,5.62
2021-09-03 14:11:35,ALGO,trade,0.000000,0.000000,0.000000,1.2353,0.00
2021-09-03 14:11:35,ETH,trade,0.000000,0.000000,0.000000,3969.0400,0.00
2021-09-03 14:11:35,MATIC,trade,0.000000,0.000000,0.000000,1.4645,0.00


## Writing code to SQLite db

In [14]:
import sqlite3 as sql

### Creating the Database
- This step will only need to be run once. 

In [26]:
# dataframe that will create the new database:

holdings_append

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-03 14:59:40,USDT,trade,2335.289008,1969.705577,365.583431,NaN,2335.29
2021-09-03 14:59:40,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-03 14:59:40,BTC,trade,0.013654,0.013654,0.000000,50714.1000,692.47
2021-09-03 14:59:40,BTC3S,trade,334.099100,120.711700,213.387400,NaN,334.10
2021-09-03 14:59:40,BTC3L,trade,128.540200,21.594900,106.945300,NaN,128.54
2021-09-03 14:59:40,ATOM,trade,0.236600,0.236600,0.000000,24.0596,5.69
2021-09-03 14:59:40,ALGO,trade,0.000000,0.000000,0.000000,1.2578,0.00
2021-09-03 14:59:40,ETH,trade,0.000000,0.000000,0.000000,3983.0000,0.00
2021-09-03 14:59:40,MATIC,trade,0.000000,0.000000,0.000000,NaN,0.00


In [27]:
conn = sql.connect('kucoin.db')

In [35]:
holdings_append.to_sql('holdings', con = conn, if_exists = 'append')

In [29]:
# Testing reading from the SQLite database I just created.

db_test = pd.read_sql('SELECT * FROM holdings', conn)
db_test

,date,currency,act_name,balance,available,holds,price,dollar_value
0,2021-09-03 14:59:40,USDT,trade,2335.289008,1969.705577,365.583431,NaN,2335.29
1,2021-09-03 14:59:40,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2,2021-09-03 14:59:40,BTC,trade,0.013654,0.013654,0.000000,50714.1000,692.47
3,2021-09-03 14:59:40,BTC3S,trade,334.099100,120.711700,213.387400,NaN,334.10
4,2021-09-03 14:59:40,BTC3L,trade,128.540200,21.594900,106.945300,NaN,128.54
5,2021-09-03 14:59:40,ATOM,trade,0.236600,0.236600,0.000000,24.0596,5.69
6,2021-09-03 14:59:40,ALGO,trade,0.000000,0.000000,0.000000,1.2578,0.00
7,2021-09-03 14:59:40,ETH,trade,0.000000,0.000000,0.000000,3983.0000,0.00
8,2021-09-03 14:59:40,MATIC,trade,0.000000,0.000000,0.000000,NaN,0.00
